In [1]:
# 데이터 다운로드 링크로 데이터를 코랩에 불러옵니다.

!wget 'https://bit.ly/3i4n1QB'

import zipfile
with zipfile.ZipFile('3i4n1QB', 'r') as existing_zip:
    existing_zip.extractall('data')

--2022-03-28 04:53:34--  https://bit.ly/3i4n1QB
Resolving bit.ly (bit.ly)... 67.199.248.11, 67.199.248.10
Connecting to bit.ly (bit.ly)|67.199.248.11|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://drive.google.com/uc?export=download&id=1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw [following]
--2022-03-28 04:53:34--  https://drive.google.com/uc?export=download&id=1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw
Resolving drive.google.com (drive.google.com)... 172.253.115.113, 172.253.115.102, 172.253.115.101, ...
Connecting to drive.google.com (drive.google.com)|172.253.115.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-10-10-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/jd2b9cp7hqv60v3a5578nak7a4gnv6f8/1648443150000/17946651057176172524/*/1emLrrpFWT8dCoj5BJb12-5QMG2-nruUw?e=download [following]
--2022-03-28 04:53:34--  https://doc-10-10-docs.googleusercontent.com/docs/securesc/ha0r

In [2]:
# 라이브러리 및 데이터 불러오기
import pandas as pd
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

train.drop('index',axis = 1 ,inplace =True)
test.drop('index',axis = 1 ,inplace =True)

In [3]:
# 데이터 확인
train.head()

,quality,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,type
0,5,5.6,0.695,0.06,6.8,0.042,9.0,84.0,0.99432,3.44,0.44,10.2,white
1,5,8.8,0.610,0.14,2.4,0.067,10.0,42.0,0.99690,3.19,0.59,9.5,red
2,5,7.9,0.210,0.39,2.0,0.057,21.0,138.0,0.99176,3.05,0.52,10.9,white
3,6,7.0,0.210,0.31,6.0,0.046,29.0,108.0,0.99390,3.26,0.50,10.8,white
4,6,7.8,0.400,0.26,9.5,0.059,32.0,178.0,0.99550,3.04,0.43,10.9,white


In [4]:
# 원핫 인코딩 (pd.get_dummies())

train_one = pd.get_dummies(train)
test_one = pd.get_dummies(test)

In [5]:
# StratifiedKFold라이브러리를 이용해 5개의 fold로 나눔.

skf = StratifiedKFold(n_splits = 5)

X = train_one.drop('quality',axis = 1)
y = train_one['quality']
cnt = 1
acc = 0
for train_idx, valid_idx in skf.split(X,y):
  
  train_data = train_one.iloc[train_idx]
  valid_data = train_one.iloc[valid_idx]

  # 모델 정의
  model = LGBMClassifier()

  # train_X 는 train_data에서 quality 를 제외한 모든 변수
  # train_y 는 train_data의 qulity 변수

  train_X = train_data.drop('quality',axis= 1)
  train_y = train_data['quality']

  # fit 메소드를 이용해 모델 학습
  model.fit(train_X,train_y)


  valid_X = valid_data.drop('quality',axis = 1)
  valid_y = valid_data['quality']

  # predict 메소드와 valid_X 데이터를 이용해 품질 예측
  pred = model.predict(valid_X)

  # 모델 정확도 출력
  print(cnt," 번째 모델 정확도 : " ,accuracy_score(pred,valid_y))
  acc += accuracy_score(pred,valid_y)
  cnt+=1

print('모델 정확도 평균 : ',acc/5)



1  번째 모델 정확도 :  0.6418181818181818
2  번째 모델 정확도 :  0.6481818181818182
3  번째 모델 정확도 :  0.6451319381255687
4  번째 모델 정확도 :  0.6223839854413102
5  번째 모델 정확도 :  0.6533212010919017
모델 정확도 평균 :  0.6421674249317562
